In [108]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, f1_score, average_precision_score
from sklearn.model_selection import train_test_split

In [109]:
# Cargar el archivo CSV
data_trasversal = pd.read_csv(r"../data/final/2_trasversal/final_dataset_trasversal.csv")
data_trasversal = data_trasversal.dropna().reset_index(drop=True)
data_trasversal.to_csv(r'../data/final/2_trasversal/4_linea/df_features_original.csv', index=False)
data_trasversal = data_trasversal.drop(columns=['conglome', 'vivienda','hogar','codperso','periodo','ocupinf'])
data_trasversal
print(data_trasversal.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122891 entries, 0 to 122890
Data columns (total 96 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   dpto                     122891 non-null  int64  
 1   dominio                  122891 non-null  int64  
 2   tuvotrabajo              122891 non-null  int64  
 3   sector_trabajador        122891 non-null  int64  
 4   categoria_trabajador     122891 non-null  int64  
 5   trabajopara              122891 non-null  float64
 6   registropersonajuridica  122891 non-null  float64
 7   cuentassunat             122891 non-null  float64
 8   pagosueldo               122891 non-null  int64  
 9   pagosalario              122891 non-null  int64  
 10  pagocomision             122891 non-null  int64  
 11  pagodestajo              122891 non-null  int64  
 12  pagosubvencion           122891 non-null  int64  
 13  pagohonorarios           122891 non-null  int64  
 14  pago

In [110]:
########## Separamos del dataset las variables que serán útiles para la línea de vulnerabilidad y que son parte del modelo, pero
########## no son parte del modelo ni son predictores. Las guardamos en un dataset aparte.


In [111]:
categorical_numeric = [
    col for col in data_trasversal.columns 
    if data_trasversal[col].dtype in ['float64', 'int64'] and data_trasversal[col].nunique() < 20
]

for col in categorical_numeric:
    data_trasversal[col] = data_trasversal[col].astype(int).astype(str)

In [112]:
categoricas = ['sexo','estadocivil','tuvotrabajo','categoria_trabajador','sector_trabajador'
               ,'trabajopara','registropersonajuridica',
               'cuentassunat','tipocontrato','numpersonastrabajo','tuvootrotrabajo','normtrabaja',
               'disponiblehorastrabajar',
               'lenguamaterna','niveleduc','asiste_educ','usointernet','padece_enfer',
               'sintoma_malestar','enfermedad','recaida','accidente','notuvoenf',
               'puestosalud','centrosalud','centroopuesto','posta','hospital_minsa','hospital_seguro',
               'hospital_ffaa','consultorio','clinica','farmacia','domicilio','otro','no_busco',
               'no_dinero','lejos','demora_mucho','noconfia','nograve','remedio_caseros','no_seguro',
               'autoreceto','notiempo','maltrato','otro_motivo','essalud','seguroprivado',
               'eps','seguroffaa','sis','tipovivienda','materialparedes','materialpisos','materialtechos',
               'vivienda_status','viviendatitulo','agua_procedencia','conexionsshh','electricidad',
               'celular','internet','pobreza','dominio','dpto','ciiu_6c','area',
               'region','onp','afp',
               'pagosueldo','pagosalario','pagocomision','pagodestajo','pagosubvencion','pagohonorarios','pagoganancianegocio','pagoagropecuario','pagopropina','pagoespecie','pagootros','pagonorecibe']
# 'antepasadosconsidera','perteneceindig','registrosunat','deseaotrotrabajo','viviadistrito','agua_potable'
df_categoricas = data_trasversal[categoricas]
df_continuas = data_trasversal.drop(columns=categoricas)

print("La cantidad de variables indep categoricas:")
print(df_categoricas.shape[1])
print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])



La cantidad de variables indep categoricas:
83
La cantidad de variables indep continuas:
13


In [113]:
# Crear dummies a partir de las variables categóricas
df_categoricas_encoded = pd.get_dummies(df_categoricas, columns=categoricas, drop_first=True, dummy_na=False)
print('La cantidad de variables indep categóricas después de haberse convertido en dummies es:')
print(df_categoricas_encoded.shape[1])

La cantidad de variables indep categóricas después de haberse convertido en dummies es:
230


In [114]:
# El dataset final de variables independientes (categoricas convertidas a dummies + continuas)
df_indep_final = pd.concat([df_categoricas_encoded, df_continuas], axis=1)
print('La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:')
print(df_indep_final.shape[1])

La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:
243


In [115]:
df_indep_final

,sexo_2,estadocivil_2,estadocivil_3,estadocivil_4,estadocivil_5,estadocivil_6,tuvotrabajo_2,categoria_trabajador_2,categoria_trabajador_3,categoria_trabajador_4,...,habitaciones,habitacionesdormir,combustible,personas_hogar,personas_ingresos,tiempotrabajo,alquiler,horastotales_sempasada,ingtrabw,ratiodep
0,True,False,False,False,True,False,False,False,True,False,...,7,4,2,4,2,23.000000,369,9,63271.0,0.500000
1,False,False,False,False,True,False,False,False,False,False,...,2,1,2,1,1,30.000000,295,47,32175.0,1.000000
2,True,False,False,False,False,True,False,False,True,False,...,6,4,2,4,4,16.833334,886,72,23945.0,1.000000
3,False,True,False,False,False,False,False,False,True,False,...,5,4,2,3,3,14.416667,1000,40,31881.0,1.000000
4,True,True,False,False,False,False,False,False,True,False,...,5,4,2,3,3,38.000000,1000,40,20712.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122886,True,False,False,False,False,True,False,False,True,False,...,2,1,2,3,2,0.583333,50,40,7107.0,0.666667
122887,False,True,False,False,False,False,False,False,False,True,...,2,1,2,4,2,30.000000,150,48,62823.0,0.500000
122888,False,True,False,False,False,False,False,False,True,False,...,6,4,2,5,3,3.000000,100,58,64075.0,0.600000
122889,True,True,False,False,False,False,False,False,True,False,...,6,4,2,5,3,17.666666,100,50,17140.0,0.600000


In [116]:
df_linea = df_indep_final.dropna()
df_linea

,sexo_2,estadocivil_2,estadocivil_3,estadocivil_4,estadocivil_5,estadocivil_6,tuvotrabajo_2,categoria_trabajador_2,categoria_trabajador_3,categoria_trabajador_4,...,habitaciones,habitacionesdormir,combustible,personas_hogar,personas_ingresos,tiempotrabajo,alquiler,horastotales_sempasada,ingtrabw,ratiodep
0,True,False,False,False,True,False,False,False,True,False,...,7,4,2,4,2,23.000000,369,9,63271.0,0.500000
1,False,False,False,False,True,False,False,False,False,False,...,2,1,2,1,1,30.000000,295,47,32175.0,1.000000
2,True,False,False,False,False,True,False,False,True,False,...,6,4,2,4,4,16.833334,886,72,23945.0,1.000000
3,False,True,False,False,False,False,False,False,True,False,...,5,4,2,3,3,14.416667,1000,40,31881.0,1.000000
4,True,True,False,False,False,False,False,False,True,False,...,5,4,2,3,3,38.000000,1000,40,20712.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122886,True,False,False,False,False,True,False,False,True,False,...,2,1,2,3,2,0.583333,50,40,7107.0,0.666667
122887,False,True,False,False,False,False,False,False,False,True,...,2,1,2,4,2,30.000000,150,48,62823.0,0.500000
122888,False,True,False,False,False,False,False,False,True,False,...,6,4,2,5,3,3.000000,100,58,64075.0,0.600000
122889,True,True,False,False,False,False,False,False,True,False,...,6,4,2,5,3,17.666666,100,50,17140.0,0.600000


In [117]:
pd.DataFrame(df_linea.columns, columns=['Variables'])


,Variables
0,sexo_2
1,estadocivil_2
2,estadocivil_3
3,estadocivil_4
4,estadocivil_5
...,...
238,tiempotrabajo
239,alquiler
240,horastotales_sempasada
241,ingtrabw


In [118]:
import joblib
import pandas as pd
from itertools import combinations

# Cargar los objetos guardados
logit = joblib.load("../output/modelos/modelo_logit.pkl")
scaler = joblib.load("../output/modelos/scaler.pkl")
selected_features = joblib.load("../output/modelos/features.pkl")

print("Características finales usadas por el modelo:")
print(selected_features)


Características finales usadas por el modelo:
['sexo_2 niveleduc_6', 'sexo_2 ciiu_6c_2', 'sexo_2 pagosalario_1', 'estadocivil_2 ratiodep', 'estadocivil_5 sector_trabajador_4', 'estadocivil_5 normtrabaja_2', 'estadocivil_5 niveleduc_8', 'estadocivil_5 enfermedad_1', 'estadocivil_5 hospital_seguro_1', 'estadocivil_5 farmacia_1', 'estadocivil_5 nograve_1', 'estadocivil_5 autoreceto_1', 'estadocivil_5 materialparedes_3', 'estadocivil_5 viviendatitulo_2', 'estadocivil_5 viviendatitulo_99', 'estadocivil_5 electricidad_2', 'estadocivil_5 region_2', 'estadocivil_5 region_3', 'estadocivil_5 pagosalario_1', 'estadocivil_5 ratiodep', 'estadocivil_6 sector_trabajador_7', 'estadocivil_6 normtrabaja_2', 'estadocivil_6 niveleduc_6', 'estadocivil_6 usointernet_2', 'estadocivil_6 enfermedad_1', 'estadocivil_6 recaida_1', 'estadocivil_6 notiempo_1', 'estadocivil_6 materialparedes_3', 'estadocivil_6 materialpisos_2', 'estadocivil_6 materialpisos_6', 'estadocivil_6 viviendatitulo_99', 'estadocivil_6 domin

In [119]:
import numpy as np

# Copiar el dataset original
df_linea_poly = df_linea.copy()

df_linea_poly = df_linea_poly.apply(pd.to_numeric, errors='coerce')


# Detectar las interacciones de segundo grado (las que contienen un espacio)
interaction_features = [f for f in selected_features if ' ' in f]

# Crear interacciones una por una
for inter in interaction_features:
    vars_ = inter.split(' ')
    # Validar que ambas columnas existan antes de multiplicar
    if all(v in df_linea_poly.columns for v in vars_):
        df_linea_poly[inter] = df_linea_poly[vars_[0]] * df_linea_poly[vars_[1]]
    else:
        print(f"⚠️ No se pudo crear {inter} porque falta alguna de {vars_}")

# Asegurar que existan todas las columnas esperadas por el modelo
missing_cols = [c for c in selected_features if c not in df_linea_poly.columns]
if missing_cols:
    print(f"⚠️ {len(missing_cols)} columnas faltantes. Se crearán con ceros.")
    for col in missing_cols:
        df_linea_poly[col] = 0

# Crear el dataset final en el mismo orden que el modelo espera
df_linea_final = df_linea_poly[selected_features]

print("✅ Dataset listo con todas las interacciones y columnas requeridas.")


C:\Users\sergio.cuadros\AppData\Local\Temp\ipykernel_6748\2636665382.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_linea_poly[inter] = df_linea_poly[vars_[0]] * df_linea_poly[vars_[1]]
C:\Users\sergio.cuadros\AppData\Local\Temp\ipykernel_6748\2636665382.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_linea_poly[inter] = df_linea_poly[vars_[0]] * df_linea_poly[vars_[1]]
C:\Users\sergio.cuadros\AppData\Local\Temp\ipykernel_6748\2636665382.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually

✅ Dataset listo con todas las interacciones y columnas requeridas.


In [120]:
df_linea_final = df_linea_poly[selected_features]


In [121]:
df_linea_final.to_csv(r'../data/final/2_trasversal/3_modelling/df_linea_final.csv', index=False)